In [5]:
##import 
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import numpy as np
import pandas as pd
import re
import psycopg2 as pg 



def linkGet():
    links_data = []
    data_frame = []
    for x in range(1,14):
        ton = urlopen(f'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page={x}')
        read_ton = soup(ton.read())

        for link in read_ton.findAll("a"):
            if "href" in link.attrs:
                links_val = link.attrs["href"]
                if links_val[:len("https")] == "https":
                    doc_array = links_val.split("/")
                    if len(doc_array) == 6:
                        if "organisations" not in doc_array:
                            links_data.append(links_val)
        new_lst = []
        for i in links_data:
            i_val = i.split("/")
            for n in range(0, len(i_val)):
                if i_val[n] == "publications":
                        new_lst.append([])
            new_lst[-1].append(i)

        for i in new_lst:
            data = []
            for n in i:
                data.append(n)
            data_frame.append(data)

    df = pd.DataFrame(data_frame)
    df.columns = ["publication_link", "Author_profile", "co-author_Profile", "co_author_profile2"]
    df.head()
    print("Get Worked")
    return df


def link2Get():
    data_frame = []
    links_books = []
    for x in range(1,14):
        ton = urlopen(f'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page={x}')
        read_ton = soup(ton.read())
        
        for link in read_ton.findAll("a"):
            if "href" in link.attrs:
                links_val = link.attrs["href"]
                if links_val[:len("https")] == "https":
                    doc_array = links_val.split("/")
                    if len(doc_array) == 6:
                        if "organisations" not in doc_array:
                            for k in doc_array[4:]:
                                links_books.append(k)


        new_lst=[]
        target = "publications"
        for i in links_books:
            if i == target:
                new_lst.append([])
            new_lst[-1].append(i)


        data_frame = []
        for i in new_lst:
            data = []
            for n in i:
                data.append(n)
            data_frame.append(data)


    df_d = pd.DataFrame(data_frame)
    df_d.columns = ["publication_index", "Publication", "person_value", "Author", "person_value2",
                    "Co-author", "person_name3", "Co-author2"]


    data_df = df_d.drop(["publication_index", "person_value", "person_value2", "person_name3"], axis=1)
    return data_df


def saveScrapDate(final_output):
    #Save Scrap data as Json file in path to be read
    json_output = r'C:\Users\USER\Desktop\WORKPLACE\Dataset\DataforUpload.json'
    output = final_output.to_json(json_output, indent = 1, orient = 'records')
    return json_output


def SendDatatoPostGre(host, database, user, password):
    #####Sending Data to PostgresL

    # Connect to the database
    conn = pg.connect(
        host = host,
        database = database,
        user = user,
        password = password
    )

    # Create a cursor
    cur = conn.cursor()

    # Load the DataFrame from a file or some other source
    df = pd.read_json(json_output)  #====> Json

    # Convert the DataFrame to a list of tuples
    data = [tuple(x) for x in df.values]

    # Create the table
    cur.execute("""
    CREATE TABLE TestTable(
        id SERIAL PRIMARY KEY,
        Publication text,
        Author text,
        CoAuthorI text,
        CoAauthorII text,
        publication_link varchar,
        Author_profile text,
        co_author_Profile text,
        co_author_profile2 text
    )
    """)

    # Insert the data into the table
    cur.executemany("""
    INSERT INTO TestTable(Publication, Author, CoAuthorI, CoAauthorII, publication_link,
           Author_profile, co_author_Profile, co_author_profile2)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, data)

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()
    return

def SearchforAuthor(final_output):
    ##### search authors or co-authors by name ######
    search_value = input("enter authors or co-authors name: ").lower()
    array_values = final_output.columns[1:4]
    for i in array_values:
        ss = final_output[final_output[i] == search_value]
        print(ss)
        return

In [ ]:
#Main Body of Project

dx = linkGet()
dy = link2Get()

final_output = pd.concat([dy, dx], axis=1)
final_output.head(10)

final_output.columns = ['Publication', 'Author', 'CoAuthorI', 'CoAauthorII', 'publication_link',
       'Author_profile', 'co_author_Profile', 'co_author_profile2']

saveScrapDate(final_output)

SearchforAuthor(final_output)

host = "localhost",
database = "postgres",
user = "postgres",
password = input("ENTER POSTGRESQL PASSWORD: ")

SendDatatoPostGre(host, database, user, password, json_output)
